# <center> IGR204 - Projet TimeX - 2017/2018

In [1]:
import os
import numpy as np
import pandas as pd
idx = pd.IndexSlice

In [2]:
# Fonctions permettant la conversions des temps depuis "hh:mm" (String) vers leur équivalent en minutes (int)
def stringToMinutes(stringTime):
    assert ":" in stringTime, "char ':' not in given parameter"
    if stringTime == ":":
        return None
    hours, minutes = stringTime.split(":", maxsplit=1)
    return int(hours)*60 + int(minutes)

# Fonctions inverse
def minutesToString(intTime):
    assert isinstance(intTime, int), "parameter must be an integer"
    hours, minutes = divmod(intTime, 60)
    return "{}:{:02d}".format(hours, minutes)

In [3]:
# Lecture des données
data_filename = "./data/original_data.csv"

df = pd.read_csv(data_filename,
                 #na_values=":",
                 #converters={'Value':stringToMinutes},
                 index_col=['ACL00', 'ACL00_LABEL', 'TIME', 'SEX', 'SEX_LABEL',
                            'DAYSWEEK', 'DAYSWEEK_LABEL', 'GEO', 'GEO_LABEL'])

df = df.drop(['UNIT', 'UNIT_LABEL'], axis=1)

df['Minutes'] = df['Value'].apply(stringToMinutes)
df = df.drop(['Value'], axis=1)

df  #.isna().sum().sum()

Minutes
ACL00   ACL00_LABEL          TIME SEX SEX_LABEL DAYSWEEK DAYSWEEK_LABEL       GEO GEO_LABEL                                                
TOTAL   Total                2000 T   Total     D1-7     All days of the week BE  Belgium                                            1440.0
                                                                              BG  Bulgaria                                           1440.0
                                                                              DE  Germany (until 1990 former territory of the FRG)   1440.0
                                                                              EE  Estonia                                            1440.0
                                                                              EL  Greece                                                NaN
                                                                              ES  Spain                                              1440.0
                                                                              FR  France                                             1440.0
                                                                              IT  Italy                                              1440.0
                                                                              LV  Latvia                                             1440.0
                                                                              LT  Lithuania                                          1440.0
                                                                              LU  Luxembourg                                            NaN
                                                                              HU  Hungary                                               NaN
                                                                              NL  Netherlands                                           NaN
                                                                              AT  Austria                                               NaN
                                                                              PL  Poland                                             1440.0
                                                                              RO  Romania                                               NaN
                                                                              SI  Slovenia                                           1440.0
                                                                              FI  Finland                                            1440.0
                                                                              UK  United Kingdom                                     1440.0
                                                                              NO  Norway                                             1440.0
                                                                              RS  Serbia                                                NaN
                                                D1-5     Monday to Friday     BE  Belgium                                            1440.0
                                                                              BG  Bulgaria                                           1440.0
                                                                              DE  Germany (until 1990 former territory of the FRG)   1440.0
                                                                              EE  Estonia                                            1440.0
                                                                              EL  Greece                                                NaN
                                                                              ES  Spain                                              1440.0
                                                                        

## Extract Countries & Activities List

In [4]:
countries = list(df.index.get_level_values(8).unique().values)
activities_Code = list(df.index.get_level_values(0).unique().values)

print("Nb Countries =", len(countries))
print("Nb Activities =", len(activities_Code))

Nb Countries = 21
Nb Activities = 56


In [5]:
## Vérification qu'il y a bien 21 valeurs (pour les 21 pays) par filtrage
for year in [2000, 2010]:
    for gender in ['M', 'F', 'T']:
        for daysweek in ['D1-5', 'D6-7', 'D1-7']:
            for acl in activities_Code:
                nb_valeurs = len(df.loc[idx[acl, :, year, gender, :, daysweek], :].values)
                assert nb_valeurs==21, "!! Il y a {} valeurs pour Y={}, gender={}, day={}".format(nb_valeurs,
                                                                                                  year,
                                                                                                  gender,
                                                                                                  daysweek)


## DataFrame des Activities

In [6]:
activities_Code = list(df.index.get_level_values(0).unique().values)
activities_label = list(df.index.get_level_values(1).unique().values)
df_activities = pd.DataFrame({"ACL":activities_Code, "Label":activities_label})
df_activities #.iloc[25:]
#df_activities.to_excel("activities_TimeUse.xlsx")

,ACL,Label
0,TOTAL,Total
1,AC0,Personal care
2,AC01,Sleep
3,AC02,Eating
4,AC03,Other and/or unspecified personal care
5,AC1_TR,"Employment, related activities and travel as p..."
6,AC1A,Main and second job and related travel
7,AC1B,Activities related to employment and unspecifi...
8,AC2,Study
9,AC21A,School and university except homework


## Select Constraint / Leisure Activities

In [7]:
leisure_media = ['AC22', 'AC52', 'AC53', 'AC733', 'AC7A', 'AC82', 'AC811', 'AC812', 'AC83']
leisure_sports_outdoors = ['AC611', 'AC6A']
leisure_social_meetings = ['AC38B', 'AC43', 'AC4-8', 'AC51A', 'AC51B', 'AC7B', 'AC9D']

total_leisure = leisure_media + leisure_sports_outdoors + leisure_social_meetings
print("NB activities leisure =", len(total_leisure))

NB activities leisure = 18


In [8]:
# print(df_activities[df_activities["ACL"].isin(leisure_media)])
# print()
# print(df_activities[df_activities["ACL"].isin(leisure_sports_outdoors)])
# print()
# print(df_activities[df_activities["ACL"].isin(leisure_social_meetings)])

In [9]:
basic_needs = ['AC0', 'AC01', 'AC02', 'AC03']
pro_study = ['AC1_TR', 'AC1A', 'AC1B', 'AC2', 'AC21A', 'AC212', 'AC9A', 'AC913', 'AC9B']
household_family = ['AC3', 'AC31A', 'AC313', 'AC321', 'AC32A', 'AC331', 'AC332',
                    'AC33A', 'AC34A', 'AC342', 'AC343', 'AC344', 'AC35', 'AC36',
                    'AC38A', 'AC37_39', 'AC41', 'AC42', 'AC936', 'AC938', 'AC9C']

total_constraint = basic_needs + pro_study + household_family
print("NB activities constraint =", len(total_constraint))

NB activities constraint = 34


In [10]:
# print(df_activities[df_activities["ACL"].isin(basic_needs)])
# print()
# print(df_activities[df_activities["ACL"].isin(pro_study)])
# print()
# print(df_activities[df_activities["ACL"].isin(household_family)])

In [11]:
# OTHERS
total_others = set(activities_Code) -  (set(total_leisure) | set(total_constraint)) # | for Union, & for Intersection
df_activities[df_activities["ACL"].isin(total_others)]

,ACL,Label
0,TOTAL,Total
46,AC4-8NSP,Unspecified leisure
54,AC90NSP,Unspecified travel
55,AC99NSP,Unspecified time use


## General Function to Compute GHI

In [12]:
dict_leisure = {'leisure_media': leisure_media,
                'leisure_sports_outdoors': leisure_sports_outdoors,
                'leisure_social_meetings': leisure_social_meetings}

dict_constraint = {'basic_needs': basic_needs,
                   'pro_study': pro_study,
                   'household_family': household_family}

In [13]:
# ['ACL00', 'ACL00_LABEL', 'TIME', 'SEX', 'SEX_LABEL', 'DAYSWEEK', 'DAYSWEEK_LABEL', 'GEO', 'GEO_LABEL']

def computeGHI_allCountries(df, dict_leisure, dict_constraint, year=2000, gender='T', daysweek='D1-7'):
    
    assert year in [2000, 2010], "year must be an integer equal to 2000 or 2010"
    assert gender in ['M', 'F', 'T'], "gender must be a String equal to 'M', 'F' or 'T'"
    assert daysweek in ['D1-5', 'D6-7', 'D1-7'], "daysweek must be a String equal to 'D1-5', 'D6-7' or 'D1-7'"
    
    # Set a new DataFrame with Country labels as index and Country Code in first column
    idx = pd.IndexSlice
    countries = list(df.index.get_level_values(8).unique().values)
    countries[2] = 'Germany'  # to replace 'Germany (until 1990 former territory of the FRG)' by 'Germany'
    df_res = pd.DataFrame(index=countries)
    df_res.index.name = 'country'
    countries_code = list(df.index.get_level_values(7).unique().values)
    df_res.insert(0, "country_code", countries_code)
    
    
    # Compute Total Time by CONSTRAINT domain
    for domain in dict_constraint.keys():
        df_res[domain] = np.zeros(len(countries))
        for acl in dict_constraint[domain]:
            df_res[domain] += df.loc[idx[acl, :, year, gender, :, daysweek], :].values.reshape(-1)

    # Compute Total Constraint
    df_res['total_constraint'] = df_res[list(dict_constraint.keys())].sum(axis = 1)
    
    
    # Compute Total Time by LEISURE domain
    for domain in dict_leisure.keys():
        df_res[domain] = np.zeros(len(countries))
        for acl in dict_leisure[domain]:
            df_res[domain] += df.loc[idx[acl, :, year, gender, :, daysweek], :].values.reshape(-1)
            
    # Compute Total Leisure
    df_res['total_leisure'] = df_res[list(dict_leisure.keys())].sum(axis = 1)
    
    
    # Compute GHI
    df_res['GHI'] = df_res['total_leisure'] / df_res['total_constraint']
    df_res['GHI'] = df_res['GHI'].round(6)
    
    # Sort by GHI
    #df_res.sort_values('GHI', ascending=False, inplace=True)
    
    return df_res

In [14]:
### Par exemple :
year=2000
gender='T'
daysweek='D1-7'

df_res = computeGHI_allCountries(df, dict_leisure, dict_constraint, year=2000, gender='T', daysweek='D1-7')
filename = "data_transposed_{}_{}_{}.csv".format(year, gender, daysweek)
#df_res.to_csv(filename, sep=',')
df_res

,country_code,basic_needs,pro_study,household_family,total_constraint,leisure_media,leisure_sports_outdoors,leisure_social_meetings,total_leisure,GHI
country,,,,,,,,,,
Belgium,BE,1316.0,430.0,418.0,2164.0,235.0,22.0,424.0,681.0,0.314695
Bulgaria,BG,1409.0,451.0,485.0,2345.0,180.0,21.0,327.0,528.0,0.225160
Germany,DE,1299.0,449.0,434.0,2182.0,203.0,27.0,454.0,684.0,0.313474
Estonia,EE,1266.0,548.0,488.0,2302.0,209.0,19.0,343.0,571.0,0.248045
Greece,EL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
Spain,ES,1336.0,514.0,415.0,2265.0,174.0,46.0,386.0,606.0,0.267550
France,FR,1418.0,469.0,NaN,1887.0,NaN,29.0,NaN,29.0,0.015368
Italy,IT,1347.0,498.0,444.0,2289.0,165.0,30.0,382.0,577.0,0.252075
Latvia,LV,1299.0,639.0,389.0,2327.0,187.0,25.0,337.0,549.0,0.235926


## Print all combination of filters into CSV files

In [20]:
for year in [2000, 2010]:
    for gender in ['M', 'F', 'T']:
        for daysweek in ['D1-5', 'D6-7', 'D1-7']:
            
            # Create table
            df_res = computeGHI_allCountries(df, dict_leisure, dict_constraint, year, gender, daysweek)
            
            # Create a new folder where save files in
            directory = "output_tables/"   #_SORTED_by_GHI/"
            if not os.path.exists(directory):
                os.makedirs(directory)
            
            # Save tables in CSV format
            filename = "data_{}_{}_{}.csv".format(year, gender, daysweek)
            df_res.to_csv(directory + filename, sep=',')
            print("created in {}  ==>  {}".format(directory, filename))

created in output_tables/  ==>  data_2000_M_D1-5.csv
created in output_tables/  ==>  data_2000_M_D6-7.csv
created in output_tables/  ==>  data_2000_M_D1-7.csv
created in output_tables/  ==>  data_2000_F_D1-5.csv
created in output_tables/  ==>  data_2000_F_D6-7.csv
created in output_tables/  ==>  data_2000_F_D1-7.csv
created in output_tables/  ==>  data_2000_T_D1-5.csv
created in output_tables/  ==>  data_2000_T_D6-7.csv
created in output_tables/  ==>  data_2000_T_D1-7.csv
created in output_tables/  ==>  data_2010_M_D1-5.csv
created in output_tables/  ==>  data_2010_M_D6-7.csv
created in output_tables/  ==>  data_2010_M_D1-7.csv
created in output_tables/  ==>  data_2010_F_D1-5.csv
created in output_tables/  ==>  data_2010_F_D6-7.csv
created in output_tables/  ==>  data_2010_F_D1-7.csv
created in output_tables/  ==>  data_2010_T_D1-5.csv
created in output_tables/  ==>  data_2010_T_D6-7.csv
created in output_tables/  ==>  data_2010_T_D1-7.csv


In [ ]:
#print("Created Files :")
#for r, d, f in os.walk(directory):
#    for file in f:
#        print(os.path.join(r, file))

### HELP:
> Si on a plusieurs activités + plusieurs pays en multiindex,  
et qu'on veut mettre un des deux en colonnes :

```python
df.unstack(level=...)
```